# CARREGAR BIBLIOTECAS

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from tqdm import tqdm


# FUNÇÕES 

In [14]:
def salvandoDadosVestibularDFtoCSV(df_,nome_arq,path='ds'):
    df_.to_csv(f'{path}/{nome_arq}.csv', sep=';',index=False)

# precisa deixar global
def urlListaClassificacaoVestibular(url):
    driver = webdriver.Chrome()
    return driver.get(url)

def popularSelect(driver,id_Xpath):
    return  Select(driver.find_element_by_id(f'{id_Xpath}'))

def criarDicionario(select_,fields):
    lista_ = [] 
    for c in range(len (select_.options)):
        if 'Selecione...' not in select_.options[c].text:
            #key_ = int(select_.options[c].get_attribute("value"))
            key_= (int(select_.options[c].get_attribute("value")) if select_.options[c].get_attribute("value").isdigit() == True  else c)
            value_= select_.options[c].text
            lista_.append({fields[0]:key_,fields[1]:value_})
    return lista_

def criarDataFrame(l):
     return pd.DataFrame(l)
    
def buscarCursosPopular(url_cursos):
    rCurso = requests.get(url_cursos)
    soupCurso = bs(rCurso.content)    
    divCursos = soupCurso.find(id = "cursos")
    lista_links=divCursos.find_all("a")
    lista_ = [{'id':245,'curso':'Gestão Empresarial - EaD'}]
    
    for c in range (len(lista_links)):
        key = int((divCursos.find_all('a')[c]['href']).split("=")[1])
        value= divCursos.find_all("a")[c].get_text()
        lista_.append({'id':key,'curso':value})
        
    return salvandoDadosVestibularDFtoCSV(criarDataFrame(lista_),'listaCursos')

def ajustarNota(detalhe_nota):
    return detalhe_nota.text.upper().split(' ')[-6]

def curso_periodo(txt):
    #recebe uma string, faz uma lógica para separar em duas partes. Devolvendo um array de 2 itens. O índice 0 é o curso e o outro o período.
    return (txt.replace(")", "")).split('(')



In [15]:
#classificacao_geral_vest_fatec = urlListaClassificacaoVestibular('https://www.vestibularfatec.com.br/classificacao/fatec.asp')
classificacao_geral_vest_fatec = webdriver.Chrome()
classificacao_geral_vest_fatec.get("https://www.vestibularfatec.com.br/classificacao/fatec.asp")

In [4]:
#PRECISA MELHORAR A LÓGICA. ANTES DE CRIAR OBJETO, VERIFICAR SE O DATAFRAME JÁ TEM CSV

In [16]:
# não precisa disso agora.
selectFatecs = Select(classificacao_geral_vest_fatec.find_element_by_id('CodFatec'))


In [17]:
listaFatecs = criarDicionario(selectFatecs,['id','fatec'])

In [6]:
len(listaFatecs)

73

In [32]:
#Criando o DataFrame a partir da lócica acima. Porém ajustar para ter a opção de 

In [8]:
#COM OS DATAFRAMES CRIADOS, AGORA REALIZO A LÓCICA DE NAVEGAÇÃO

# ROBO EM OPERAÇÃO

In [18]:
dfFatec = criarDataFrame(listaFatecs)

In [8]:
dfFatec.head()

,id,fatec
0,71,Adamantina - Fatec Adamantina
1,3,Americana - Fatec Americana - Ministro Ralph B...
2,35,Araçatuba - Fatec Araçatuba - Prof. Fernando A...
3,67,Araraquara - Fatec Araraquara
4,68,Araras - Fatec Araras


In [11]:
#POPULAR O DATAFRAMECURSOS
buscarCursosPopular('https://www.vestibularfatec.com.br/unidades-cursos/?q=')


In [19]:
dfCursos = pd.read_csv('ds/listaCursos.csv',sep=';')

In [10]:
dfCursos.head()

,id,curso
0,245,Gestão Empresarial - EaD
1,247,Agroindústria
2,75,Agronegócio
3,76,Alimentos
4,266,Análise de Processos Agroindustriais


In [20]:
resultado_vestibular_da_fatec = []
resultado_vestibular_da_fatec2 = [] #Organiza cada fatec em uma array.


'''
lista_info_vestibular_fatec = {
    'cod_curso':'' ,
    'nome_curso':'',
    'cod_instituicao':'',
    'instituicao':'',
    'ano':'', 
    'semestre':'',
    'periodo':'',
    'qtde_vagas':'',
    'qtde_inscrito':'',
    'demanda':'',
    'nota_corte':'',
    'nota_maxima':''
}
'''
#inf
'''
cod_curso-> Sitema da Calculadora
cod_instituicao -> Sistema da Calculadora
ano -> ano do vestibular
semeste-> semestre do vestibular
periodo -> da busca (UPPER())
ROBO DEMANDA
qtde_vagas-> preencher através de outro robo
qtde_inscrito->
demanda-> realizar o cálculo
nota_corte -> nota min
nota_maxima -> nota max


'''


'\ncod_curso-> Sitema da Calculadora\ncod_instituicao -> Sistema da Calculadora\nano -> ano do vestibular\nsemeste-> semestre do vestibular\nperiodo -> da busca (UPPER())\nROBO DEMANDA\nqtde_vagas-> preencher através de outro robo\nqtde_inscrito->\ndemanda-> realizar o cálculo\nnota_corte -> nota min\nnota_maxima -> nota max\n\n\n'

# PASSO A PASSO DA LÓGICA
SERÁ FEITO UMA NAVEGAÇÃO SEGUINDO A SEGUINTE REGRA:
    -> OS DATAFRAMES ESTÃO POPULADOS
    A CADA FATEC DA LISTA, VISITAR A PÁGINA DOS CURSOS QUE ELA OFERECERAM. ENTRAR EM CADA UMA DAS OPÇÕES E IR PARA A TELA DE VERIFICAR LISTA DE CHAMADA.

In [21]:
for i in tqdm(range(len (dfFatec.head()))):
    resultado_por_fatec=[]
    id_ = dfFatec.loc[i,'id']
    fatec_ = dfFatec.loc[i,'fatec']
    #CARREGA UMA LISTA COM OS DADOS DO SELECT DA PAGINA. RECEBE OS DADOS DA FATEC
    selectFatecs = Select(classificacao_geral_vest_fatec.find_element_by_id('CodFatec'))
    #NAVEGA NA FATEC DA VEZ. ELE FAZ ESSE CICLO, SEGUINDO O DATAFRAME. PREENCHIDO EM OUTRO MOMENTO OU PELA FUNÇÃO OU PELO CSV
    selectFatecs.select_by_visible_text(fatec_)
    #APÓS SELECIONAR NO COMBO A FATEC - CLICA NO BOTÃO PARA IR PARA A PÁGINA DA ESCOLHA DO CURSO
    classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()
    #SEGURA A PÁGINA PARA QUE ELA POSSA ESTAR 100% CARREGADA PARA A PRÓXIMA ETAPA
    time.sleep(4)
    #PREENCHE A LISTA DE CURSOS OFERECIDAS NA FATEC SELECIONADA ANTERIRMENTE
    selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))
    #CRIA UM DICIONARIO PARA FACILITAR A NAVEGAÇÃO, ATRAVÉS DE UMA FUNÇÃO 
    listaDeCursos = criarDicionario(selectCursos,['id','curso'])
    #SEGURA A PÁGINA
    time.sleep(2)

    #print(f'quantidade de cursos {len(selectCursos.options)}')
    #ENTRA EM UM LOOP PARA VISITAR TODOS OS CURSOS DA FATEC SELECIONADA
    for c in range(len (listaDeCursos)):
        lista_info_vestibular_fatec = {}
        #RECARREGA O OBJETO PARA NÃO PERDER O PONTEIRO. DÁ PARA MELHORAR ESSE TRECHO
        selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))
        print(f'{fatec_} - {listaDeCursos[c]["curso"]}')
        #ESSE VERIFICADOR IMPEDE QUE A OPÇÃO SELECIONE SEJA ITERADA
        if 'Selecione...' not in listaDeCursos[c]['curso']:
            #selectCursos.select_by_visible_text(listaDeCursos[c]['curso'])
            #RECUPERO O ID DO CURSO E PASSO PARA A FUNÇÃO DO SELENIUM PARA SELECIONAR O ITEM NO SELECT
            idC = str(listaDeCursos[c]['id'])
            selectCursos.select_by_value(idC)
            #REALIZO O CLICK PARA CHEGAR NA PRÓXIMA PÁGINA
            classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()   
            #SEGURO UM POUCO A PÁGINA PARA QUE CONSIGA CARREGAR. VERIFICAR SE EXISTE METÓDO MELHOR PARA AGUARDAR
            time.sleep(2)   
            #PEGO A URL ATUAL E CONCATENO COM AS INFORMAÇÕES DE IDENTIFICAÇÃO DA FATEC E CURSO.
            #ESSE TRECHO PRECISA SER ADAPTADO PARA CASO O USUÁRIO JÁ SEJA REDIRECIONADO PARA A PAGINA DESTINO. ISSO ACONTECE CASO NÃO TENHA LISTA DE CONVOCAÇÃO DE 2 CHAMADA
            url_lista_1chamada = classificacao_geral_vest_fatec.current_url
            #cod_curso = int(selectCursos.options[c].get_attribute("value"))
            url_lista_1chamada = (f'{url_lista_1chamada}?codfatec={id_}&codescolacurso={idC}&o=1')
            #REALIZO O CLICK PARA CONSULTAR A TABELA DE CLASSIFICAÇÃO COM AS NOTAS
            classificacao_geral_vest_fatec.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/ul/li[1]/a').click()   
            time.sleep(2)
            #SELECIONO A TABELA COM O RESULTADO
            tabela_ =classificacao_geral_vest_fatec.find_element_by_css_selector('table.table')
            #AQUI ESTÁ UMA SACADA. COMO TEM CURSOS COM MUITOS INSCRITOS E AS VAGAS NO MÁXIMO SÃO DE 80 VAGAS. FAÇO UMA SELEÇÃO APENAS DOS 80 PRIMEIROS
            resultado80 = tabela_.find_elements_by_css_selector('tbody tr:nth-child(-n + 100)')
            #SEGURO A PÁGINA PARA QUE POSSA CARREGAR
            time.sleep(5)
            #A NOTA MÁXIMA SEMPRE SERÁ O PRIMEIRO REGISTRO. POR ISSO ATRIBUIÇÃO DIRETA.
            nota_max= ajustarNota(resultado80[0])
            #AGORA PRECISO DESCOBRIR A NOTA MINÍMA(CORTE) OU SEJA O ÚLTIMO CONVOCADO
            nota_min = 0
            #FACO UMA ITERAÇÃO DENTRO DA LISTA COM OS 80 PRIMEIROS. PODERIAM AJUSTAR PARA PERCORRER DE FORMA MULTIPLO DE 5. POIS O MÍNIMO DE VAGAS É 30
            for i, linha in enumerate(resultado80):
#                lista_info_vestibular_fatec = {}
                #DENTRO DESSA ITERAÇÃO, TENHO CERTEZA QUE PEGAREI APENAS O CONVOCADO COM OO IF, SEMPRE ATUALIZANDO A NOTA MIN A CADA PASSAGEM. ITERROMPO, QUANDO A CONDIÇÃO DO IF FOR FALSE
                if ('CONVOCADO' in linha.text.upper()):
                    nota_min = ajustarNota(resultado80[i])
                else:
                    break            
            print(f'NOTA MÁXIMA = {nota_max} | NOTA DE CORTE = {nota_min}')
            #NESTA ETAPA JÁ POSSUO OS DADOS PARA CRIAR UM OBJETO COMOS RESULTADOS DESTE CURSO, PERIODO E DESTA FATEC
            lista_info_vestibular_fatec['nome_curso'] = curso_periodo(listaDeCursos[c]["curso"])[0]#curso
            lista_info_vestibular_fatec['instituicao'] = fatec_.strip()
            lista_info_vestibular_fatec['ano']= 2021
            lista_info_vestibular_fatec['semestre']= 1
            lista_info_vestibular_fatec['periodo'] = curso_periodo(listaDeCursos[c]["curso"])[1]#periodo
            lista_info_vestibular_fatec['nota_corte'] = nota_min
            lista_info_vestibular_fatec['nota_maxima'] = nota_max
            #ARMAZENO O RESULTADO EM UMA LISTA. CRIO DUAS MANEIRAS DISTINTAS
            resultado_vestibular_da_fatec.append(lista_info_vestibular_fatec)            
            resultado_por_fatec.append(lista_info_vestibular_fatec)
            #RETORNO DUAS VEZES PARA RECOMEÇAR O PASSO A PASSO
            classificacao_geral_vest_fatec.back()
            classificacao_geral_vest_fatec.back()
            resultado_vestibular_da_fatec2.append(resultado_por_fatec)
    classificacao_geral_vest_fatec.back()
    time.sleep(1)


  0%|          | 0/5 [00:00<?, ?it/s]

Adamantina - Fatec Adamantina - Ciência de Dados (Noite)
NOTA MÁXIMA = 55,00 | NOTA DE CORTE = 35,75
Adamantina - Fatec Adamantina - Gestão Comercial (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 39,55


 20%|██        | 1/5 [00:34<02:19, 34.83s/it]

Americana - Fatec Americana - Ministro Ralph Biasi - Análise e Desenvolvimento de Sistemas (Tarde)
NOTA MÁXIMA = 50,00 | NOTA DE CORTE = 34,38
Americana - Fatec Americana - Ministro Ralph Biasi - Análise e Desenvolvimento de Sistemas (Manhã)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 41,25
Americana - Fatec Americana - Ministro Ralph Biasi - Gestão Empresarial (EaD)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 46,75
Americana - Fatec Americana - Ministro Ralph Biasi - Gestão Empresarial (Manhã)
NOTA MÁXIMA = 53,68 | NOTA DE CORTE = 38,78
Americana - Fatec Americana - Ministro Ralph Biasi - Gestão Empresarial (Noite)
NOTA MÁXIMA = 55,00 | NOTA DE CORTE = 46,75
Americana - Fatec Americana - Ministro Ralph Biasi - Jogos Digitais (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 41,25
Americana - Fatec Americana - Ministro Ralph Biasi - Logística (Manhã)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 34,38
Americana - Fatec Americana - Ministro Ralph Biasi - Logística (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 

 40%|████      | 2/5 [03:20<03:41, 73.96s/it]

...

 60%|██████    | 3/5 [04:09<02:13, 66.60s/it]

Araraquara - Fatec Araraquara - Gestão Comercial (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 44,00
Araraquara - Fatec Araraquara - Gestão Empresarial (Manhã)
NOTA MÁXIMA = 55,00 | NOTA DE CORTE = 39,55
Araraquara - Fatec Araraquara - Segurança da Informação (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 46,75


 80%|████████  | 4/5 [04:59<01:01, 61.47s/it]

Araras - Fatec Araras - Desenvolvimento de Software Multiplataforma (Noite)
NOTA MÁXIMA = 56,50 | NOTA DE CORTE = 40,15
Araras - Fatec Araras - Gestão Empresarial (Noite)
NOTA MÁXIMA = 55,00 | NOTA DE CORTE = 41,25


100%|██████████| 5/5 [05:34<00:00, 66.83s/it]


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=88.0.4324.150)


In [114]:
resultado_tabela = webdriver.Chrome()
resultado_tabela.get("https://www.vestibularfatec.com.br/classificacao/lista.asp?codfatec=1&codescolacurso=1999&o=1")

In [140]:
#tabela_ = resultado_tabela.find_elements_by_css_selector('.table.table-bordered.table-striped')
#tabela_ = resultado_tabela.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[2]/table')

#tabela_ = resultado_tabela.find_elements_by_class_name('table')

tabela_ =resultado_tabela.find_element_by_css_selector('table.table')




In [191]:
#-Processo para verificar a posição da NOTA e SITUAÇÃO
pos_nota = 0
pos_situacao = 0
      
for count,cell in enumerate(tabela_.find_elements_by_css_selector('thead th')):
    print(cell.text,count)
    if (cell.text =='Situação'):
        pos_situacao = count
    if (cell.text =='Nota'):
        pos_nota = count


Class. 0
Nº Inscrição 1
Nome 2
Nota 3
Afrodesc. 4
Esc. Pública 5
Português 6
Matemática 7
Situação 8


In [151]:
demanda_por_curso = webdriver.Chrome()
demanda_por_curso.get("https://www.vestibularfatec.com.br/demanda/")

In [80]:
select_ano_sem =demanda_por_curso.find_element_by_name('ano-sem')

In [81]:
todos_anos_sem=select_ano_sem.find_elements_by_tag_name("option")

In [82]:
for option in todos_anos_sem:
    print("Value is: %s" % option.get_attribute("value"))
    option.click()

Value is: Selecione...
Value is: 20211
Value is: 20202
Value is: 20201
Value is: 20192
Value is: 20191
Value is: 20182
Value is: 20181
Value is: 20172
Value is: 20171
Value is: 20162
Value is: 20161
Value is: 20152
Value is: 20151
Value is: 20142
Value is: 20141
Value is: 20132
Value is: 20131
Value is: 20122
Value is: 20121
Value is: 20112
Value is: 20111
Value is: 20102
Value is: 20101
Value is: 20092
Value is: 20091
Value is: 20082
Value is: 20081
Value is: 20072
Value is: 20071


In [84]:
todos_anos_sem[1].click()

In [152]:
selectOptionsAnoSem = Select(demanda_por_curso.find_element_by_name('ano-sem'))
selectOptionsAnoSem.select_by_value('20211') #informar o ano e semestre que quer capturar.
frmdemanda = demanda_por_curso.find_element_by_id("formDemanda")
frmdemanda.find_element_by_css_selector('button.btn'). submit()

In [153]:
selectFatecsDemanda = Select(demanda_por_curso.find_element_by_id('FATEC'))
listaFatecsDemanda = criarDicionario(selectFatecsDemanda,['id','fatec'])
dfFatecDemanda = criarDataFrame(listaFatecsDemanda)
resydfFatecDemanda.head()

,id,fatec
0,1,Fatec Adamantina
1,2,Fatec Americana - Ministro Ralph Biasi
2,3,Fatec Araçatuba - Prof. Fernando Amaral de Alm...
3,4,Fatec Araraquara
4,5,Fatec Araras


In [154]:
armazenando_resultado_demanda=[]
for i in tqdm(range(len (dfFatecDemanda.head(1)))):
    parcial_demanda = {}
    id_ = dfFatecDemanda.loc[i,'id']
    fatec_ = dfFatecDemanda.loc[i,'fatec']
    selectFatecs = Select(demanda_por_curso.find_element_by_id('FATEC'))
    selectFatecs.select_by_visible_text(fatec_)
    print(f'{fatec_} - {i+1}')

    demanda_por_curso.find_element_by_css_selector('button.btn'). submit()  
    
    tabela_demanda = demanda_por_curso.find_element_by_css_selector('table.table')
    results=[]
    for count,cell in enumerate(tabela_demanda.find_elements_by_css_selector('tbody td')):
        
        results.append(cell)
        print(cell.text,count)    
    time.sleep(1)
    x = {'fatec':fatec_,'demanda'result:results}

    #demanda_por_curso.back()


  0%|          | 0/1 [00:00<?, ?it/s]

Fatec Adamantina - 1
Ciência de Dados 0
Noite 1
68 2
35 3
1,94 4
Gestão Comercial 5
Noite 6
92 7
35 8
2,63 9


100%|██████████| 1/1 [00:01<00:00,  1.95s/it]
